#### Nama : Amin Fahri 
#### Kode Peserta : PYTN-KS01-002

Anggota Kelompok :
1. Amin Fahri (PYTN-KS01-002)
2. Jessica Athalia Rieuwpassa (PYTN-KS01-005)
3. Ninda Nurismiranda (PYTN-KS01-010)

# Final Project 1

## Perkenalan

Objektif yang ingin dicapai dari final project 1 ini yaitu mengevaluasi konsep regression sebagai berikut :
- Mampu memahami konsep regression dengan Linear Regression
- Mampu mempersiapkan data untuk digunakan dalam model Linear Regression
- Mampu mengimplementasikan Linear Regression untuk membuat prediksi

Dataset yang digunakan yaitu **Uber and Lyft Dataset Boston, MA** yang berisi record data perjalanan taksi uber dan lyft di Boston, MA. Dataset ini terdiri dari 57 kolom dan 693.071 data. Dataset diperoleh dari kaggle dengan link sebagai berikut https://www.kaggle.com/brllrb/uber-and-lyft-dataset-boston-ma

Dataset ini mengandung banyak nilai NA dan outliers maka cocok untuk dilakukan data cleaning dan preprocessing. Selain itu, dataset ini juga terdiri dari banyak kolom (variabel) sehingga sangat cocok untuk membuat model regresi linier untuk melihat pola antara variabel-variabel yang berbeda, serta membuat suatu prediksi berdasarkan model regresi linier. 

Pada final project 1 ini akan dibuat prediksi harga taksi Uber dan Lyft (cab price), sehingga variabel price akan menjadi variabel dependen (variabel prediktor) dan beberapa variabel lainnya yang memiliki korelasi dengan variabel price akan menjadi variabel independen.

## Import Pustaka

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pickle

In [3]:
pd.set_option("display.max.columns", None)

## Data Loading

In [4]:
df = pd.read_csv("rideshare_kaggle.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'rideshare_kaggle.csv'

In [ ]:
df.head()

Dataset ini memiliki 57 atribut. Atribut dalam dataset ini antara lain :
1. id : unique identifier for each column
2. timestamp : unix timestamp
3. hour : hour of the day
4. day : day of the week
5. month : month in a year
6. datetime : date value
7. Timezone
8. source : initial source of the ride
9. destination : destination of the ride
10. cab_type : the type of cab
11. product_id
12. name
13. price
14. distance
15. surge_multiplier
16. . . . dan lainnya

## Data Cleaning 

In [ ]:
# Print the dimension of data set
df.shape

In [ ]:
df.columns

### Cek data types

In [ ]:
#Melihat tipe objek masing-masing kolom
df.info()

Dataset terdiri dari column dengan tipe object, int, dan float

In [ ]:
# menghapus data column yang tidak diperlukan
data = df.drop(['id','timestamp','datetime','long_summary','apparentTemperatureHighTime','apparentTemperatureLowTime',
                  'apparentTemperatureLowTime','windGustTime','sunriseTime','sunsetTime','uvIndexTime','temperatureMinTime',
                 'temperatureMaxTime','apparentTemperatureMinTime','temperatureLowTime','apparentTemperatureMaxTime'
                 ,'latitude','longitude'],axis = 1)
print(data.shape)
data.head()

### Cek data temperature dengan price

In [ ]:
# membuat data frame baru yang terdiri dari variabel price dan temperature
data_new = data[['price','temperature','apparentTemperature','temperatureHigh','temperatureLow','apparentTemperatureHigh',
                'apparentTemperatureLow','temperatureMin','temperatureHighTime','temperatureMax','apparentTemperatureMin','apparentTemperatureMax']]
data_new.head()

In [ ]:
#membuat plot heatmap untuk melihat korelasi antara variabel price dan temperature
plt.figure(figsize=(10,10))
sns.heatmap(data_new.corr(),annot=True, mask=np.triu(data_new.corr()))

Dari plot di atas dapat dilihat antara variabel prediktor (price) dengan variabel-variabel temperature memiliki korelasi yang sangat lemah karena nilai-nilainya hampir 0 sehingga variabel-variabel temperature dapat dihapus.

In [ ]:
# drop data temperature features
data_new = data[['temperature','apparentTemperature','temperatureHigh','temperatureLow','apparentTemperatureHigh',
                'apparentTemperatureLow','temperatureMin','temperatureHighTime','temperatureMax','apparentTemperatureMin','apparentTemperatureMax']]
data = data.drop(data_new.columns,axis=1)
data.head()

### Cek Data Categorical

In [ ]:
cat_col = data.select_dtypes(include=['object','category']).columns.tolist()
data_cat = data[cat_col]
data_cat.head()

In [ ]:
# jumlah unique values pada data_cat
data_cat.nunique()

In [ ]:
# unique values pada data_cat
for col in data_cat:
    print(f'{col} : {data_cat[col].unique()}')
    print()

Dari cek unique values terlihat bahwa variable timezone hanya memiliki 1 unique value yaitu 'America/New_York' sehingga variabel timezone dapat dihapus.

In [ ]:
data_cat['product_id'].value_counts()

Variabel product_id terdiri lebih dari 50% value dengan junk data (contoh : 6f72dfc5-27f1-42e8-84db-ccc7a75f6969) sehingga variabel product_id lebih baik dihapus.

In [ ]:
# drop data timezone and product_id
data = data.drop(['product_id','timezone'],axis=1)
data.head()

### Cek Data Numerical

In [ ]:
num_col = data.select_dtypes(include=['int64','float64']).columns.tolist()
data_num = data[num_col]
data_num.columns

sebagian besar column dengan tipe numerical (int64 dan float64) merupakan column dengan variabel climate. Selanjutnya akan dicek korelasi antara variabel-variabel climate dengan variabel price.

### Cek data climate dengan price

In [ ]:
# membuat data frame dengan variabel climate dan price
cli_col = ['price','precipIntensity', 'precipProbability', 'humidity', 'windSpeed',
       'windGust', 'visibility', 'dewPoint', 'pressure', 'windBearing',
       'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'moonPhase',
       'precipIntensityMax']
data_cli = data[cli_col]
data_cli.head()

In [ ]:
#membuat plot heatmap untuk melihat korelasi pada data_cli
plt.figure(figsize=(12,12))
sns.heatmap(data_cli.corr(),annot=True, mask=np.triu(data_cli.corr()))

Dari plot di atas dapat dilihat bahwa korelasi antara variabel prediktor (price) dengan variabel-variabel climate memiliki korelasi yang lemah karena nilainya hampir 0, sehingga variabel-variabel temperature tersebut dapat dihapus.

In [ ]:
# drop data climate features
cli_col = ['precipIntensity', 'precipProbability', 'humidity', 'windSpeed',
       'windGust', 'visibility', 'dewPoint', 'pressure', 'windBearing',
       'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'moonPhase',
       'precipIntensityMax']
data = data.drop(cli_col,axis=1)
data.head()

In [ ]:
data.shape

## Data Preprocessing

### Data encoding

Pada tahap data encoding dilakukan untuk variabel-variabel dengan tipe object dan category dengan tujuan untuk mengubah dari nilai object/category menjadi numeric (0 dan 1) sehingga variabel-variabel tersebut dapat diproses lebih lanjut.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_col = data.select_dtypes(include=['object','category']).columns.tolist()
print(cat_col)

# menjalankan OneHotEncoder dan menggabungkan data frame original (data) dengan data frame hasil encoding (enc_df)
for col in cat_col:
    encoder = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(encoder.fit_transform(data[[col]]).toarray())
    enc_df.columns = encoder.get_feature_names([col])
    data = data.drop(col, axis=1)
    data = pd.concat([data, enc_df], axis=1)

In [ ]:
data.columns

In [ ]:
data.head()

Data di atas merupakan hasil penggabungan dataframe original dan data hasil encoding yang akan digunakan untuk tahap selanjutnya. Pada data di atas terdapat variabel-variabel baru yaitu kolom source, destination, name, summary, dan icon. Maka tahap selanjutnya yaitu cek korelasi kolom-kolom tersebut dengan variabel prediktor (price).

### Cek data source dengan price

In [ ]:
# membuat dataframe dengan variabel price dan source
src_col = ['price','source_Back Bay', 'source_Beacon Hill', 'source_Boston University',
       'source_Fenway', 'source_Financial District', 'source_Haymarket Square',
       'source_North End', 'source_North Station',
       'source_Northeastern University', 'source_South Station',
       'source_Theatre District', 'source_West End', 'destination_Back Bay']
data_src = data[src_col]
data_src.head()

In [ ]:
# membuat plot heatmap untuk melihat korelasi pada data_src
plt.figure(figsize=(12,12))
sns.heatmap(data_src.corr(),annot=True, mask=np.triu(data_src.corr()))

Dari plot di atas dapat dilihat bahwa antara variabel prediktor (price) tidak memiliki korelasi dengan variabel-variabel source karena nilai korelasinya negatif dan ada juga yang hampir 0, sehingga variabel-variabel source tersebut dapat dihapus.

In [ ]:
src_cols = ['source_Back Bay', 'source_Beacon Hill', 'source_Boston University',
       'source_Fenway', 'source_Financial District', 'source_Haymarket Square',
       'source_North End', 'source_North Station',
       'source_Northeastern University', 'source_South Station',
       'source_Theatre District', 'source_West End']
data = data.drop(src_cols,axis=1)

In [ ]:
print(data.shape)
data.head()

### Cek data destination dengan price

In [ ]:
# membuat dataframe dengan variabel price dan destination
dst_col = ['price','destination_Back Bay',
       'destination_Beacon Hill', 'destination_Boston University',
       'destination_Fenway', 'destination_Financial District',
       'destination_Haymarket Square', 'destination_North End',
       'destination_North Station', 'destination_Northeastern University',
       'destination_South Station', 'destination_Theatre District',
       'destination_West End']
data_dst = data[dst_col]
data_dst.head()

In [ ]:
# membuat plot heatmap untuk melihat korelasi pada data_dst
plt.figure(figsize=(12,12))
sns.heatmap(data_dst.corr(),annot=True, mask=np.triu(data_dst.corr()))

Dari plot di atas dapat dilihat bahwa antara variabel prediktor (price) tidak memiliki korelasi dengan variabel-variabel destination karena nilai korelasinya negatif dan ada juga yang hampir 0, sehingga variabel-variabel destination tersebut dapat dihapus.

In [ ]:
# drop source dan destination column value
dst_cols = ['destination_Back Bay','destination_Back Bay',
       'destination_Beacon Hill', 'destination_Boston University',
       'destination_Fenway', 'destination_Financial District',
       'destination_Haymarket Square', 'destination_North End',
       'destination_North Station', 'destination_Northeastern University',
       'destination_South Station', 'destination_Theatre District',
       'destination_West End']
data = data.drop(dst_cols,axis=1)

In [ ]:
print(data.shape)
data.head()

###  Cek data summary dengan price

In [ ]:
# membuat dataframe dengan variabel price dan summary
summ_cols = ['price','short_summary_ Drizzle ', 'short_summary_ Foggy ',
       'short_summary_ Light Rain ', 'short_summary_ Mostly Cloudy ',
       'short_summary_ Overcast ', 'short_summary_ Partly Cloudy ',
       'short_summary_ Possible Drizzle ', 'short_summary_ Rain ','short_summary_ Clear ']
data_summ = data[summ_cols]
data_summ.head()

In [ ]:
#membuat plot heatmap untuk melihat korelasi pada data_summ
plt.figure(figsize=(12,12))
sns.heatmap(data_summ.corr(),annot=True, mask=np.triu(data_summ.corr()))

Dari plot di atas dapat dilihat bahwa antara variabel prediktor (price) tidak memiliki korelasi dengan variabel-variabel sumary karena nilai korelasinya negatif dan ada juga yang hampir 0, sehingga variabel-variabel summary tersebut dapat dihapus.

In [ ]:
# drop data column summary
summ_cols = ['short_summary_ Drizzle ', 'short_summary_ Foggy ',
       'short_summary_ Light Rain ', 'short_summary_ Mostly Cloudy ',
       'short_summary_ Overcast ', 'short_summary_ Partly Cloudy ',
       'short_summary_ Possible Drizzle ', 'short_summary_ Rain ','short_summary_ Clear ']
data = data.drop(summ_cols,axis=1)

In [ ]:
print(data.shape)
data.head()

### Cek data icon dengan price

In [ ]:
# membuat dataframe dengan variabel price dan icon
ico_cols = ['price','icon_ clear-day ', 'icon_ clear-night ', 'icon_ cloudy ', 'icon_ fog ',
       'icon_ partly-cloudy-day ', 'icon_ partly-cloudy-night ',
       'icon_ rain ']
data_ico = data[ico_cols]
data_ico.head()

In [ ]:
# membuat plot heatmap untuk melihat korelasi pada data_ico
plt.figure(figsize=(12,12))
sns.heatmap(data_ico.corr(),annot=True, mask=np.triu(data_ico.corr()))

Dari plot di atas dapat dilihat bahwa antara variabel prediktor (price) tidak memiliki korelasi dengan variabel-variabel icon karena nilai korelasinya negatif dan ada juga yang hampir 0, sehingga variabel-variabel icon tersebut dapat dihapus.

In [ ]:
# drop data column icon
ico_cols = ['icon_ clear-day ', 'icon_ clear-night ', 'icon_ cloudy ', 'icon_ fog ',
       'icon_ partly-cloudy-day ', 'icon_ partly-cloudy-night ',
       'icon_ rain ']
data = data.drop(ico_cols,axis=1)

In [ ]:
print(data.shape)
data.head()

### Cek data name dengan price

In [ ]:
# membuat dataframe dengan variabel price dan name
name_cols = ['price','name_Black', 'name_Black SUV',
       'name_Lux', 'name_Lux Black', 'name_Lux Black XL', 'name_Lyft',
       'name_Lyft XL', 'name_Shared', 'name_Taxi', 'name_UberPool',
       'name_UberX', 'name_UberXL', 'name_WAV']
data_name = data[name_cols]
data_name.head()

In [ ]:
# membuat plot heatmap untuk melihat korelasi pada data_name
plt.figure(figsize=(12,12))
sns.heatmap(data_name.corr(),annot=True, mask=np.triu(data_name.corr()))

Dari plot di atas dapat dilihat bahwa beberapa variabel name memiliki korelasi positif dengan variabel price mencapai nilai korelasi 0.5, sehingga variabel name dapat menjadi variabel yang berpengaruh terhadap variabel price.

### Cek data remaining column dengan price

In [ ]:
# membuat dataframe dengan variabel price dan variabel lainnya yang tersisa
rem_cols =['price','hour', 'day', 'month', 'distance', 'surge_multiplier','cab_type_Lyft','cab_type_Uber']
data_rem = data[rem_cols]
data_rem.head()

In [ ]:
# membuat plot heatmap untuk melihat korelasi pada data_rem
plt.figure(figsize=(12,12))
sns.heatmap(data_rem.corr(),annot=True, mask=np.triu(data_rem.corr()))

Dari plot di atas dapat dilihat bahwa variabel yang cukup memiliki korelasi dengan variabel price yaitu **surge_multiplier** dan **distance** dengan nilai korelasi **0.24** dan **0.35**, sedangkan variabel lainnya tidak memiliki korelasi atau memiliki korelasi yang lemah dengan variabel price karena nilai korelasinya negatif ataupun hampir 0 sehingga variabel lainnya dapat dihapus.

In [ ]:
data = data.drop(['month','day','hour','cab_type_Lyft','cab_type_Uber'],axis=1)
print(data.shape)
data.head()

### Cek Missing Value Pada Dataset

In [ ]:
#Mengecek missing value dari dataset
data.isnull().sum()

Terdapat 55095 missing value pada data price. Sehingga untuk mengatasinya maka akan dihapus nilai NA pada data price dengan cara `.dropna` 

In [ ]:
# handling missing value
data = data.dropna(subset = ['price']).reset_index()

In [ ]:
# cek kembali missing value
data.shape
data.isnull().sum()

Dapat dilihat bahwa sudah tidak terdapat missing value pada data, maka dapat dilanjutkan ke tahap selanjutnya yaitu cek outliers.

## Explorasi Data Analysis

### Variabel Dependen

In [ ]:
sns.boxplot(data['price'])

Dari boxplot di atas terlihat bahwa terdapat banyak bulatan hitam, artinya data price mengandung banyak outlier. Agar model dapat bekerja dengan baik nantinya maka outliers tersebut perlu dihilangkan dengan cara menghapus nilai data price yang melebihi batas atas(max_threshold).

In [ ]:
max_threshold = data['price'].quantile(0.99)
max_threshold

Batas atas quantile yaitu 42.5

In [ ]:
# melihat data price dengan nilai lebih dari max_threshold
data[data['price']>max_threshold]

In [ ]:
# drop data price yang memiliki nilai lebih dari max_threshold atau lebih dari 42.5
out = np.where(data['price']>42.5)
print(out[0])
print(np.count_nonzero(np.where(data['price']>42.5)))

In [ ]:
data.drop(out[0], inplace = True)
data.shape

In [ ]:
# cek kembali outliers
sns.boxplot(data['price'])

Dari box plot di atas dapat dilihat bahwa data price sudah tidak mengandung outliers.

### Variabel Independen

In [ ]:
# membuat data frame yang berisi variabel price, distance, dan surge multiplier
data2 = data[['price','distance','surge_multiplier']]
data2.head()

In [ ]:
data2.info()

In [ ]:
data2.describe()

Untuk variabel distance :
- rata-rata jarak perjalanan yaitu 2,17
- jarak terdekat yang ditempuh yaitu 0,02 sedangkan jarak terjauh yang ditemputh yaitu 7,86

Untuk variabel surge multiplier :
- data surge multiplier bernilai dari rentang 1 - 3
- rata-rata surge multiplier yaitu 1

In [ ]:
# melihat distribusi dari variabel distance
f = plt.figure(figsize=(12,4))

f.add_subplot(1,2,1)
data2['distance'].plot(kind='kde')

f.add_subplot(1,2,2)
plt.boxplot(data2['distance'])
plt.show()

- Density dari distribusi variabel distance berada di sekitar angka 0 - 4
- Dari boxplot menunjukkan terdapat outliers pada variabel distance

In [ ]:
# melihat distribusi dari variabel surge_multiplier
f = plt.figure(figsize=(12,4))

f.add_subplot(1,2,1)
sns.countplot(data2['surge_multiplier'])

f.add_subplot(1,2,2)
plt.boxplot(data2['surge_multiplier'])
plt.show()

- Dari countplot terlihat bahwa sebagian besar data surge multiplier memiliki nilai 1, hanya sedikit data yang memiliki nilai 1,25 - 3
- Boxplot menunjukan terdapat outlier karena sebagian besar data bernilai 1, sehingga nilai lainnya dianggap outliers

In [ ]:
# melihat hubungan antara independent dan dependent variabel
plt.figure(figsize=(10,8))
sns.pairplot(data=data, x_vars=['distance','surge_multiplier'], y_vars=['price'], size=5, aspect=0.75)
plt.show()

Dari scatterplot di atas dapat dilihat bahwa polanya naik artinya variabel distance dan surge_multiplier memiliki korelasi positif dengan variabel price

In [ ]:
data2.corr().style.background_gradient().set_precision(2)

- Dari tabel korelasi di atas dapat dilihat bahwa variabel distance dan surge multiplier memiliki korelasi positif dengan variabel price dengan nilai 0.32 dan 0.14
- Variabel distance dan surge multiplier mempengaruhi price

## Pendefinisian Model

Setelah melalui tahap data cleaning dan preprocessing maka diperoleh data yang akan digunakan untuk pelatihan model selanjutnya. Dari hasil preprocessing, variabel yang memiliki korelasi atau berpengaruh terhadap variabel prediktor (price) yaitu variabel distance, surge_multiplier, dan name. Sehingga variabel-variabel tersebut yang akan digunakan dalam membentuk model linear regression untuk memprediksi price. Berikut datanya :

In [ ]:
data.head()

Split data menjadi data training dan testing dengan ratio 70:30

In [ ]:
from sklearn.model_selection import train_test_split
y = data['price'].values
X = data.drop(columns=['index','price'],axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

## Pelatihan Model

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model = model.fit(X_train,y_train)

`.fit()` untuk menghitung nilai optimal dari bobot 𝑏₀ dan 𝑏₁ pada model linear regression.

In [ ]:
# model intercept
print('intercept:', model.intercept_)

In [ ]:
data_column = data.drop(['index','price'], axis=1)

In [ ]:
# model slope
data_column = data.drop(['index','price'], axis=1)
coef_dict = {
    'features' : data_column.columns,
    'coef_value' : model.coef_
}
coef = pd.DataFrame(coef_dict, columns=['features', 'coef_value'])
coef

Intercept dan Slope merupakan atribut dari `model` dengan keterangan :
- `.intercept_` :  𝑏0 
- `.coef_` : 𝑏1, 𝑏2, . . ., 𝑏n

In [ ]:
y_pred = model.predict(X_test)
print('predicted response: ', y_pred, sep='\n')

## Evaluasi Model

In [ ]:
r_sq = model.score(X_train,y_train)
print('coefficient of determination:', r_sq)

- Untuk mengetahui apakah model memberikan hasil prediksi yang baik yaitu dengan nilai coefficient of determination (𝑅²). 

- Dari `.score()` diperoleh nilai 𝑅² sebesar 0.9332386099913654 atau sebesar 93,32% maka dapat dikatakan model bekerja dengan baik karena variabel distance, surge_multiplier, dan name efektif untuk memprediksi variabel price.

- Dapat diartikan pula bahwa variabel distance, surge_multiplier, dan name merupakan faktor-faktor yang mempengaruhi price sebesar 93,32%, sedangkan 6,68% sisanya dipengaruhi oleh faktor lain.

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
rootmeansq = np.sqrt(mse)
print(mse)
print(rootmeansq)

Root Mean Squared Error (RMSE) adalah suatu metode evaluasi yang didasarkan pada total kuadratis dari simpangan antara hasil model dengan hasil observasi. Nilai RMSE menunjukkan seberapa besar error yang dimiliki model dibandingkan dengan data riil/aktual.

Model regresi di atas memiliki nilai RMSE = 2.2601751143350115 sehingga error dari model yaitu sekitar 2.26%, karena nilai errornya sangat kecil artinya nilai data hasil prediksi dekat dengan nilai data aktual sehingga dapat dikatakan bahwa model cukup akurat untuk memprediksi harga taksi.

In [ ]:
pickle.dump(model, open('model_linreg.pkl','wb'))

## Prediction

####  <span style='background:yellow'>Follow  these instructions before predicting the price:</span> 
<hr>

-  **For distance**: Enter distance value using (ex : 2)

- **For Surge_multiplier** : <font color = 'red'>Enter Surge Multiplier value from 1 to 3</font>

- **for Cab Name**:  <font color = 'blue'>Put value "1" for the cab name that you want to predict and put value "0" for the rest of the cab name</font>

Misalkan prediksi harga taksi dengan kriteria sebagai berikut :
1. distance = 2.5
2. surge multiplier = 1
3. Cab Name = Lux Black XL

In [ ]:
# prediksi harga taksi
# model.predict(distance, surge_multiplier, black, black SUV, lux, lux black, Black XL, Lyft, Lyft XL, Shared, Taxi, UberPool, UberX, UberXL, WAV)

model.predict([[2.5,1,0,0,0,0,1,0,0,0,0,0,0,0,0]])

Harga taksi Lux Black XL untuk perjalanan dengan distance 2.5 dan surge multiplier 1 diprediksi sebesar $31.87

## Kesimpulan

1. Pada project ini dilakukan prediksi harga taksi uber dan lyft dengan variabel dependen yaitu 'price', serta variabel independen yaitu 'distance', 'surge multiplier', dan 'name'


2. Variabel name merupakan data category sehingga menggunakan OneHotEncoding sehingga terbagi menjadi beberapa kolom yaitu 'name_Black', 'name_Black SUV', 'name_Lux', 'name_Lux Black', 'name_Lux Black XL', 'name_Lyft', 'name_Lyft XL', 'name_Shared', 'name_Taxi', 'name_UberPool', 'name_UberX', 'name_UberXL', 'name_WAV'


3. Beberapa variabel name tersebut memiliki korelasi positif dengan variabel price mencapai nilai korelasi 0.5, sehingga variabel name merupakan variabel yang berpengaruh terhadap variabel price


4. Variabel distance dan surge multiplier juga berpengaruh terhadap memiliki korelasi positif dengan variabel price dengan nilai koreasi 0.32 dan 0.14


5. Diperoleh model regresi linier dari data dengan intercept (𝑏0) = -2042273534122.41 dan slope (𝑏1, 𝑏2, . . ., 𝑏n) sebagai berikut =

6. Koefisien determinasi (𝑅²) dari model yaitu 0.9332386099913654 atau sebesar 93,32% yang berarti bahwa variabel distance, surge_multiplier, dan name merupakan faktor-faktor yang mempengaruhi price sebesar 93,32%, sedangkan 6,68% sisanya dipengaruhi oleh faktor lain. Karena nilai 𝑅² nya besar maka model bekerja dengan baik karena variabel distance, surge multiplier, dan name efektif untuk memprediksi variabel price.


7. Nilai Root Mean Squared Error (RMSE) dari model yaitu 2.2601751143350115 sehingga errornya sekitar 2.26%, karena nilai errornya sangat kecil artinya nilai data hasil prediksi dekat dengan nilai data aktual sehingga dapat dikatakan bahwa model cukup akurat untuk memprediksi harga taksi.